# Setup

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Standard library imports
from pydantic import BaseModel
from copy import deepcopy
import threading
import warnings
import queue
import json
import os

warnings.filterwarnings("ignore")
JWT_SECRET_API = !echo $(aws --profile "chat-prod_ro" secretsmanager get-secret-value --secret-id "arn:aws:secretsmanager:us-west-2:242659714806:secret:shared/cresta-server-jwt_secret-VDn5My" --query SecretString --output text) # type: ignore
os.environ["JWT_SECRET_API"] = json.loads(JWT_SECRET_API[0])["jwt-secret"]
os.environ["CONFIG_SERVICE_ADDR"] = "auth.chat-prod.internal.cresta.ai:443"
os.environ["CONFIG_USE_SECURE_CHANNEL"] = "true"

# Third party imports

from sentence_transformers import SentenceTransformer, util
from retry import retry

# Greyparrot imports
from greyparrot.llm.prompting import prompts as prompts_utils
from greyparrot.conversations.db import ConversationsDBConn
from greyparrot.multi_tenancy.v3_config import V3Config
from greyparrot.conversations.utils import get_chats
from greyparrot.chats_common import PartialChat
from greyparrot.common import get_logger


logger = get_logger(__name__)

# Local imports
from llm_proxy_client import LLMProxyDevClient

In [4]:
embedder = SentenceTransformer("all-mpnet-base-v2")

In [5]:
customer_id = "brinks"
profile_id = "care-voice"
usecase_id = "care-voice"
language_code = "en-US"

In [6]:
def get_chats_with_ids(chat_ids: list[str]) -> list[PartialChat]:
    customer_name: str = V3Config.short_name_from_ids(customer_id, profile_id)
    conv_db_conn: ConversationsDBConn = ConversationsDBConn.from_customer_name(customer_name)
    chats: list[PartialChat] = conv_db_conn.get_detailed_chats(
        customer_id=customer_id,
        profile_id=profile_id,
        usecase_id=usecase_id,
        language_code=language_code,
        conversation_ids=chat_ids,
        is_dev_user=False,
    )
    return chats

In [7]:
LLM_ENGINE: str = "gpt-4o-mini"
CONCURRENCY: int = 10

In [8]:
@retry(tries=1, delay=60, backoff=2, logger=logger)
def chat_completion(**kwargs):
    return LLMProxyDevClient("openai").beta.chat.completions.parse(**kwargs)

# Discovery

In [9]:
# TODO remove this after fixing speaker_role flips
flips: dict[str, str] = {"agent": "visitor", "visitor": "agent"}

In [10]:
def chat_to_prompt_text(chat: PartialChat) -> str:
    return "\n".join([
        f"{prompts_utils.speaker_role_str_for_prompts(flips[m.speaker_role.value]).capitalize()}: {m.text}"
        for m in chat.messages
    ])

In [11]:
SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY = """### Context and data description
You are a conversation analyst working for Brinks Home Security Call Center.

You will be given 1 conversation at a time. Each conversation is between a Brinks Call Center Agent and a Customer. Your primary goal is to extract workflows of steps which the Agent takes in **the given conversation** to help resolve the Customer's needs related to their home security system and services.

For each workflow, return:
- Product: Must be one of the following official Brinks products: (see below for some example products)

  PANELS & KEYPADS:
  * Security Panel (GE model)
  * Security Panel (Honeywell VISTA)
  * VISTA Security Panel
  * GoControl Security Panel
  * GoControl Panel
  * IQ4 Keypad
  * Secondary Keypad
  * Security Keypad
  * Smartbox

  CAMERAS:
  * Security Camera (ADCV723)
  * Front Door Camera
  * SkyBell Doorbell Camera
  * Doorbell Camera
  * Garage Camera
  * Patio Camera
  * Indoor Camera

  SENSORS:
  * Glass Break Sensor
  * Glass Break Detector
  * Garage Door Sensor
  * Window Sensor
  * Front Door Sensor
  * Motion Detector
  * Door/Window Sensor
  * Dining Room Window Sensor
  * Honeywell Smoke Detector
  * Living Room Smoke Detector
  * SRF433 Carbon Monoxide Monitor
  * Carbon Monoxide Detector
  * Fire Sensor
  * Hardwired Sensors
  * Honeywell Wireless Contact Sensor

  DIGITAL PRODUCTS:
  * Alarm.com Account
  * Mobile App
  * Smart Lock

  SERVICES:
  * Alarm Monitoring Service
  * Installation Service
  * Battery Replacement Service
  * Tech Support Services
  * Virtual Technician Appointment
  * Service Appointment
  * Field Service Appointment
  * Brinks HomePlus Plan
  * Brinks Home Plus Warranty

  ACCOUNT & BILLING:
  * Account Management
  * Billing Service
  * Home Security Account
  * Brinks Account Services
  * Brinks Home Security Account

  OTHER EQUIPMENT:
  * Transformer
  * Yard Sign
  * Alarm Permit

- Issue: Specific customer problem (e.g., False Alarms, Device Offline, Camera Not Recording, App Login Issues, Billing Questions, Account Changes)
- Steps: Detailed troubleshooting or resolution steps the Agent follows

Make sure to only use the exact product names from the list above, not general categories or variations.
Make sure the issue clearly describes the exact problem the customer is experiencing.
Make sure the steps are detailed and follow Brinks' standard operating procedures.
Make sure to only return the agent troubleshooting steps, not the customer's requests or other information.

Common scenarios include:
- Security system troubleshooting (panel issues, sensor malfunctions, connectivity problems)
- Camera and video recording issues
- Account management (billing, autopay, contact updates)
- Mobile app and portal assistance
- Service changes (moving, upgrading, cancellation)
- Installation and maintenance appointments

**Important**: There could be more than 1 workflow in a single conversation. There could also be no workflows in a single conversation. The workflows will be used to create troubleshooting guides to address similar customer needs in the future."""

In [12]:
class Flow(BaseModel):
    product: str
    issue: str
    steps: list[str]

class Flows(BaseModel):
    flows: list[Flow]

In [13]:
def discover_flow_in_chat(chat: PartialChat,
                          llm_engine: str = LLM_ENGINE) -> list[Flow]:
    logger.info(f"Discovering Agent flow in chat {chat.chat_name}")
    messages: list[dict[str, str]] = [
        {
            "role": "system",
            "content": SYSTEM_PROMPT_AGENT_WORKFLOW_DISCOVERY
        },
        {
            "role": "user",
            "content": chat_to_prompt_text(chat)
        },
    ]

    return chat_completion(model=llm_engine,
                            messages=messages,
                            response_format=Flows).choices[0].message.parsed.flows

In [14]:
def extract_flows_from_chats(chats: list[PartialChat],
                                concurrency: int=10) -> dict[str, list[Flow]]:
    lock: threading.Lock = threading.Lock()
    indexes: queue.Queue = queue.Queue()

    workflows: dict[str, list[Flow]] = {}
    for idx in range(len(chats)):
        indexes.put(idx)

    def workflow_labeler_worker():
        while True:
            try:
                idx = indexes.get(block=False)
            except queue.Empty:
                return
            chat = chats[idx]
            try:
                extracted_workflows: list[Flow] = discover_flow_in_chat(chat)
                with lock:
                    workflows[str(chat)] = extracted_workflows
                    if len(workflows) % 10 == 0:
                        print(f"Workflows from {len(workflows)} chats extracted!")
            except Exception as e:
                logger.warning(e, str(chat))
            indexes.task_done()

    logger.info(
        f"Starting processing {len(chats)} chats with {concurrency} workers")
    workers = [
        threading.Thread(target=workflow_labeler_worker)
        for _ in range(concurrency)
    ]
    # Start all workers
    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    logger.info(f"Finished processing all {len(chats)} chats")

    return workflows

In [15]:
test_chat: PartialChat = get_chats_with_ids(["0843c54c-6487-45ce-946a-cc6257484f54"])[0]
workflows: list[Flow] = discover_flow_in_chat(test_chat)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 14:59:41", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 14:59:43", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


{"message": "Discovering Agent flow in chat 0843c54c-6487-45ce-946a-cc6257484f54", "name": "__main__", "asctime": "2025-02-14 14:59:46", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "MainThread", "status": "INFO"}


In [16]:
print(chat_to_prompt_text(test_chat))

Customer: Technical support
Agent: Thank you for being the best part of Brinks Home. My name is Juan. Who do I have the pleasure of speaking with?
Customer: Debbie. Deborah Terryya.
Agent: Good morning, missus Deborah. How can I assist you?
Customer: Okay. On December 16th, I had Brinks installed. I had one. Camera put on my front door, and I had one put on the tireport, and then I went out of town for a couple weeks. And only my carport has been working since December 6th eight. And I clicked on the troubleshoot for my front door, and it just keeps coming up that there's a problem. I mean, I did I did everything it told me to do. I unplugged the power cord, and told me to wait 5 minutes. And it was trying to find what the problem was. And it is still not responding my front door.
Agent: The front door. The other
Customer: Which would be by my court
Agent: Okay. Gotcha. Let me get into the account real quick. And let me just double check what is happening to that sensor.
Customer: Yeah

In [17]:
print(f"Found {len(workflows)} workflows\n")
for i, flow in enumerate(workflows):
    print(f"[Agent Workflow #{i + 1}]")
    print(f"Product: {flow.product}")
    print(f"Issue: {flow.issue}")
    print("Steps:", end="\n* ")
    print("\n* ".join(flow.steps))
    if i < len(workflows) - 1:
        print("-" * 100)

Found 1 workflows

[Agent Workflow #1]
Product: Security Camera
Issue: Camera Not Recording
Steps:
* Obtain customer account information including phone number and verbal password.
* Verify the camera model and account setup.
* Instruct the customer to identify the LED color on the front door camera.
* If no light is visible, instruct the customer to unplug the power supply from the front door camera.
* Wait 30 seconds to 1 minute before reconnecting the power supply to reset the camera.
* Instruct the customer to plug the power supply back into the front door camera.
* Ask the customer to check the LED color again.
* If no light appears, instruct the customer to test the power outlet with another device (e.g., phone charger) to confirm it's working.
* If the outlet is working, conclude that the power supply for the camera might be faulty and provide the customer with the installation service number for further assistance.


# 200 Recent Chats

In [18]:
data_set = get_chats(
    'brinks-care-voice', 
    '2024-06-01',
    '2024-12-01',
    200,)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 14:59:49", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 14:59:49", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 14:59:49", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


200it [00:11, 17.53it/s]
{"message": "Caching to cache_chats-6cdad05445414399627e4b4e59113f84.pkl", "name": "cache-utils", "asctime": "2025-02-14 15:00:03", "levelname": "INFO", "filename": "cache.py", "funcName": "wrapper", "threadName": "MainThread", "status": "INFO"}
{"message": "Saving dataset to cache_chats-6cdad05445414399627e4b4e59113f84.pkl", "name": "dataset-common", "asctime": "2025-02-14 15:00:03", "levelname": "INFO", "filename": "common.py", "funcName": "save", "threadName": "MainThread", "status": "INFO"}


In [19]:
sampled_chat_ids = list(set([
    chat.chat_name
    for chat in (item if not isinstance(item, tuple) else item[0] for item in data_set)
    if hasattr(chat, 'chat_name')
]))
print(len(sampled_chat_ids))

200


# Agent Workflow Discovery

In [20]:
sampled_chats = get_chats_with_ids(sampled_chat_ids)

{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 15:00:04", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}
{"message": "use case id not set, fallback to use profile id care-voice", "name": "greyparrot.common.customers", "asctime": "2025-02-14 15:00:04", "levelname": "WARNING", "filename": "customers.py", "funcName": "__init__", "threadName": "MainThread", "status": "WARNING"}


cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice-FKrGHU -r
cmd: cresta-cli connstring -i voice-prod voice-prod brinks-care-voice -r


In [21]:
workflows = extract_flows_from_chats(sampled_chats)

{"message": "Starting processing 199 chats with 10 workers", "name": "__main__", "asctime": "2025-02-14 15:00:07", "levelname": "INFO", "filename": "<ipython-input-14-e12d352d6e64>", "funcName": "extract_flows_from_chats", "threadName": "MainThread", "status": "INFO"}
{"message": "Discovering Agent flow in chat 00cc27ec-673a-4873-8d08-662c2a957f75", "name": "__main__", "asctime": "2025-02-14 15:00:07", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-6 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 01124bf9-4f07-4bf1-a86d-30c4fc6278d6", "name": "__main__", "asctime": "2025-02-14 15:00:07", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-7 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 03c417dd-cab6-43eb-bf66-6228bcb7927c", "name": "__main__", "asc

Workflows from 10 chats extracted!


{"message": "Discovering Agent flow in chat 1bdfc2d5-f21a-4d5f-9f10-7c61fa63f117", "name": "__main__", "asctime": "2025-02-14 15:00:12", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-13 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 1cf067d8-2a3f-489d-88b0-b196277b9989", "name": "__main__", "asctime": "2025-02-14 15:00:13", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-9 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 1d1e8350-f454-4c6c-bf21-fa297464f6f9", "name": "__main__", "asctime": "2025-02-14 15:00:13", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 1de2fbb0-0fd1-46f3

Workflows from 20 chats extracted!


{"message": "Discovering Agent flow in chat 2a577a5d-ead0-45ee-8a21-d7096fbaabbc", "name": "__main__", "asctime": "2025-02-14 15:00:14", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 2b5c3071-5a33-4db9-a6c9-5f9e7b71bc20", "name": "__main__", "asctime": "2025-02-14 15:00:15", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-11 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 2cf9c0a9-7962-4380-b329-7bcfc7f799ab", "name": "__main__", "asctime": "2025-02-14 15:00:15", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-9 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat 2d391f90-ce7c-4653

...

{"message": "Discovering Agent flow in chat eee368bc-7a19-4a59-a079-ac9d65a587b6", "name": "__main__", "asctime": "2025-02-14 15:00:54", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat ef843f0a-c659-4af7-9dda-b9fc14980e36", "name": "__main__", "asctime": "2025-02-14 15:00:54", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-15 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat efc02a9d-467d-4166-8e0f-7ee3a0a90fdf", "name": "__main__", "asctime": "2025-02-14 15:00:54", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-7 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat efe14bd3-50a1-4532

Workflows from 180 chats extracted!


{"message": "Discovering Agent flow in chat f3611706-6a1c-4efc-a11b-7051d41d086d", "name": "__main__", "asctime": "2025-02-14 15:00:56", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-15 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat f3b8cf83-cab2-4d56-b258-7f47dc680f05", "name": "__main__", "asctime": "2025-02-14 15:00:56", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-10 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat f5165b8a-691d-4d3a-8a4f-206577235511", "name": "__main__", "asctime": "2025-02-14 15:00:56", "levelname": "INFO", "filename": "<ipython-input-13-ea4193a4eca2>", "funcName": "discover_flow_in_chat", "threadName": "Thread-14 (workflow_labeler_worker)", "status": "INFO"}
{"message": "Discovering Agent flow in chat f62dd888-a2de-401

Workflows from 190 chats extracted!


{"message": "Finished processing all 199 chats", "name": "__main__", "asctime": "2025-02-14 15:01:02", "levelname": "INFO", "filename": "<ipython-input-14-e12d352d6e64>", "funcName": "extract_flows_from_chats", "threadName": "MainThread", "status": "INFO"}


In [22]:
sum([len(w) for w in workflows.values()])

251

In [23]:
all_workflows = []
for w in workflows.values():
    all_workflows.extend(w)

# Grouping Workflows

In [24]:
def group_workflows_by_product(workflows: list[Flow], embedder: SentenceTransformer, semantic_threshold: float) -> dict[str, list[Flow]]:
    
    products = [w.product for w in workflows]
    embeddings = embedder.encode(products, convert_to_tensor=True)
        
    product_groups, solo_products = [], []
    for idx, _ in enumerate(products):
        existing_groups = [group for group in product_groups if idx in group]
        if existing_groups:
            assert len(
                existing_groups) == 1, "A workflow should only be in 1 group"
            continue

        scores = util.cos_sim(embeddings[idx:idx + 1],
                              embeddings).cpu().tolist()[0]
        matches = [
            i for i, score in enumerate(scores) if score > semantic_threshold
        ]
        filtered_matches = [i for i in matches if i != idx]
        if filtered_matches:
            outstanding_groups = []
            new_group = filtered_matches + [idx]
            for group in product_groups:
                if set(filtered_matches).intersection(set(group)):
                    new_group.extend(group)
                else:
                    outstanding_groups.append(group)
            product_groups = outstanding_groups + [list(set(new_group))]
        else:
            solo_products.append(idx)

    print(f"Found {len(solo_products)} # of Solo Products.")
    print(f"Found {len(product_groups)} # Groups of Products.")

    # Create mapping of indices to product names
    idx_to_product = {i: p for i, p in enumerate(products)}
    
    retval = {}

    for group in product_groups:
        group_key = tuple(set([idx_to_product[idx] for idx in group]))
        retval[group_key] = [workflows[idx] for idx in group]

    for idx in solo_products:
        retval[(workflows[idx].product,)] = [workflows[idx]]

    return retval

In [25]:
def group_workflows_by_issues(workflows: list[Flow], embedder: SentenceTransformer, semantic_threshold: float) -> dict[tuple[str], list[Flow]]:
    
    issues = [w.issue for w in workflows]
    embeddings = embedder.encode(issues, convert_to_tensor=True)
        
    issue_groups, solo_issues = [], []
    for idx, _ in enumerate(issues):
        existing_groups = [group for group in issue_groups if idx in group]
        if existing_groups:
            assert len(
                existing_groups) == 1, "A workflow should only be in 1 group"
            continue

        scores = util.cos_sim(embeddings[idx:idx + 1],
                              embeddings).cpu().tolist()[0]
        matches = [
            i for i, score in enumerate(scores) if score > semantic_threshold
        ]
        filtered_matches = [i for i in matches if i != idx]
        if filtered_matches:
            outstanding_groups = []
            new_group = filtered_matches + [idx]
            for group in issue_groups:
                if set(filtered_matches).intersection(set(group)):
                    new_group.extend(group)
                else:
                    outstanding_groups.append(group)
            issue_groups = outstanding_groups + [list(set(new_group))]
        else:
            solo_issues.append(idx)

    print(f"Found {len(solo_issues)} # of Solo Issues.")
    print(f"Found {len(issue_groups)} # Groups of Issues.")

    # Create mapping of indices to product names
    idx_to_issue = {i: p for i, p in enumerate(issues)}
    
    retval = {}

    for group in issue_groups:
        group_key = tuple(set([idx_to_issue[idx] for idx in group]))
        retval[group_key] = [workflows[idx] for idx in group]

    for idx in solo_issues:
        retval[(workflows[idx].issue,)] = [workflows[idx]]

    return retval

In [26]:
tree = group_workflows_by_product(all_workflows, embedder, semantic_threshold=0.7)

Found 16 # of Solo Products.
Found 20 # Groups of Products.


In [27]:
for product in tree.keys():
    tree[product] = group_workflows_by_issues(tree[product], embedder, semantic_threshold=0.7)

Found 6 # of Solo Issues.
Found 2 # Groups of Issues.
Found 23 # of Solo Issues.
Found 4 # Groups of Issues.
Found 19 # of Solo Issues.
Found 4 # Groups of Issues.
Found 7 # of Solo Issues.
Found 1 # Groups of Issues.
Found 0 # of Solo Issues.
Found 1 # Groups of Issues.
Found 1 # of Solo Issues.
Found 1 # Groups of Issues.
Found 2 # of Solo Issues.
Found 1 # Groups of Issues.
Found 6 # of Solo Issues.
Found 1 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 1 # of Solo Issues.
Found 1 # Groups of Issues.
Found 13 # of Solo Issues.
Found 6 # Groups of Issues.
Found 2 # of Solo Issues.
Found 1 # Groups of Issues.
Found 1 # of Solo Issues.
Found 1 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 20 # of Solo Issues.
Found 7 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 2 # of Solo Issues.
Found 0 # Groups of Issues.
Found 4 # of Solo Issues

In [28]:
leafs = 0
for product in tree.keys():
    for issue in tree[product].keys():
        leafs += 1
print(f"Found {leafs} leafs in the tree")

Found 164 leafs in the tree


In [29]:
"""
{
    "camera": [camera_flow1, camera_flow2, camera_flow3],
    "alarm": [alarm_flow4, alarm_flow5, alarm_flow6],
    "TV": [tv_flow1]
}
"""

"""
{
    "camera": {
        "issue1": [camera_issue1_flow1, camera_issue1_flow2, camera_issue1_flow3],
        "issue2": [camera_issue2_flow1, camera_issue2_flow2, camera_issue2_flow3],
    },
    "alarm": {
        "issue1": [alarm_issue1_flow1, alarm_issue1_flow2, alarm_issue1_flow3],
        "issue2": [alarm_issue2_flow1, alarm_issue2_flow2, alarm_issue2_flow3],
    },
}
"""

"""
{
    "camera": {
        "issue1": troubleshooting_steps,
        "issue2": troubleshooting_steps,
    },
    "alarm": {
        "issue1": troubleshooting_steps,
        "issue2": troubleshooting_steps,
    },
}
"""

'\n{\n    "camera": {\n        "issue1": troubleshooting_steps,\n        "issue2": troubleshooting_steps,\n    },\n    "alarm": {\n        "issue1": troubleshooting_steps,\n        "issue2": troubleshooting_steps,\n    },\n}\n'

# Generate Troubleshooting Guide for each leaf in the tree

In [30]:
class TroubleshootingGuide(BaseModel):
    steps: list[str]

In [31]:
def concatenate_flows(flows: list[Flow]) -> str:
    conversations = []
    for i, flow in enumerate(flows):
        conversation = f"Conversation {i+1}:\n" + "\n".join(f"* {step}" for step in flow.steps)
        conversations.append(conversation)
    return "\n\n".join(conversations)


In [32]:
GENERATE_TROUBLESHOOTING_GUIDE_PROMPT = """\
You are a conversation analyst working for Brinks Home Security Call Center.

You will be given the troubleshooting steps taken by a call center agent of 1 or more conversations at a time. Your primary goal is to generate an overall troubleshooting guide for the given product and issue.

The troubleshooting guide must consider all scenarios and steps from the given conversations. Do not produce any steps that are not present in the given conversations. Only produce an overall troubleshooting guide after gaining information from all the conversations.

You must produce an ordered list of steps that the agent can follow to help resolve the customer's needs related to the product and issue.
"""

In [33]:
def generate_troubleshooting_guide(product: str, issue: str, llm_engine: str = LLM_ENGINE) -> str:
    messages: list[dict[str, str]] = [
        {
            "role": "system",
            "content": GENERATE_TROUBLESHOOTING_GUIDE_PROMPT
        },
        {
            "role": "user",
            "content": concatenate_flows(tree[product][issue])
        },
    ]

    return chat_completion(model=llm_engine,
                            messages=messages,
                            response_format=TroubleshootingGuide).choices[0].message.parsed.steps

In [34]:
def generate_troubleshooting_guides(tree: dict[str, dict[str, list[Flow]]], llm_engine: str = LLM_ENGINE, concurrency: int=10) -> None:
    lock: threading.Lock = threading.Lock()
    indexes: queue.Queue = queue.Queue()
    
    # Create flat list of (product, issue) pairs to process
    tasks = []
    for product in tree.keys():
        for issue in tree[product].keys():
            tasks.append((product, issue))
            
    for idx in range(len(tasks)):
        indexes.put(idx)

    def troubleshooting_guide_worker():
        while True:
            try:
                idx = indexes.get(block=False)
            except queue.Empty:
                return
                
            product, issue = tasks[idx]
            try:
                guide = generate_troubleshooting_guide(product, issue, llm_engine)
                with lock:
                    tree[product][issue] = guide
                if idx % 10 == 0:
                    print(f"Generated guides for {idx} product/issue pairs")
            except Exception as e:
                logger.warning(f"Error generating guide for {product}/{issue}: {e}")
            indexes.task_done()

    logger.info(f"Starting guide generation for {len(tasks)} product/issue pairs")
    workers = [
        threading.Thread(target=troubleshooting_guide_worker)
        for _ in range(concurrency)
    ]
    for worker in workers:
        worker.start()
    for worker in workers:
        worker.join()
    logger.info("Finished generating all guides")
    print("Finished generating all guides")

In [35]:
troubleshooting_guide = deepcopy(tree)

In [36]:
generate_troubleshooting_guides(troubleshooting_guide)

{"message": "Starting guide generation for 164 product/issue pairs", "name": "__main__", "asctime": "2025-02-14 15:01:03", "levelname": "INFO", "filename": "<ipython-input-34-f882fd4f0c39>", "funcName": "generate_troubleshooting_guides", "threadName": "MainThread", "status": "INFO"}


Generated guides for 0 product/issue pairs
Generated guides for 10 product/issue pairs
Generated guides for 20 product/issue pairs
Generated guides for 30 product/issue pairs
Generated guides for 40 product/issue pairs
Generated guides for 50 product/issue pairs
Generated guides for 60 product/issue pairs
Generated guides for 70 product/issue pairs
Generated guides for 80 product/issue pairs
Generated guides for 100 product/issue pairs
Generated guides for 90 product/issue pairs
Generated guides for 110 product/issue pairs
Generated guides for 120 product/issue pairs
Generated guides for 130 product/issue pairs
Generated guides for 140 product/issue pairs
Generated guides for 150 product/issue pairs
Generated guides for 160 product/issue pairs


{"message": "Finished generating all guides", "name": "__main__", "asctime": "2025-02-14 15:01:56", "levelname": "INFO", "filename": "<ipython-input-34-f882fd4f0c39>", "funcName": "generate_troubleshooting_guides", "threadName": "MainThread", "status": "INFO"}


Finished generating all guides


In [37]:
for product in troubleshooting_guide.keys():
    print(f"Product: {product}")
    print(f"{'-'*100}\n{'-'*100}")
    for issue in troubleshooting_guide[product].keys():
        print(f"**Issue: {issue}**")
        print("\n".join(troubleshooting_guide[product][issue]))
        print("-"*100)
    print("-"*100)

Product: ('Alarm.com Account',)
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
**Issue: ('App Login Issues',)**
Verify the customer's identity by confirming their address and code word.
Reconfirm the customer's first and last name.
Check the account status and details related to the email address used for login.
Verify the email address on file for the account.
Determine if the customer is locked out of their account; if so, unlock the account.
Ask if the customer or anyone they know is using the same username.
Confirm if two-factor authentication is present and enabled.
If applicable, reset the password for the account and send a password reset email to the customer.
Guide the customer to set up two-factor authentication again after the password reset.
Assist the customer in verifying the received verification code for logging in.
I

In [38]:
# Function to export troubleshooting guide to Markdown
def export_to_markdown(troubleshooting_guide, output_file="troubleshooting_guide.md"):
    with open(output_file, "w") as f:
        for product in troubleshooting_guide.keys():
            f.write(f"# Product: {product}\n")
            for issue in troubleshooting_guide[product].keys():
                # Write issue and steps
                f.write(f"## **Issue: {issue}**\n\n")
                f.write("\n".join([f"- {step}" for step in troubleshooting_guide[product][issue]]) + "\n")

# Example: Export the existing troubleshooting_guide
export_to_markdown(troubleshooting_guide)


In [39]:
for product in troubleshooting_guide.keys():
    print(f"Product: {product}")

Product: ('Alarm.com Account',)
Product: ('Security Panel (GE model)', 'Security Panel (Honeywell VISTA)', 'Alarm Panel', 'Security Panel', 'Security Control Panel', 'VISTA Security Panel')
Product: ('Security Cameras', 'Front Door Camera', 'Cameras', 'SkyBell Doorbell Camera', 'Security Camera', 'Doorbell Camera')
Product: ('Alarm Monitoring Service', 'Alarm Service', 'Alarm System', 'Alarm Response Center', 'Alarm System (equipment still present)', 'Monitoring Service')
Product: ('Installation Appointment',)
Product: ('Glass Break Sensor', 'Glass Break Detector')
Product: ('Keypad', 'Secondary Keypad', 'Security Keypad', 'IQ4 Keypad')
Product: ('Mobile App',)
Product: ('Smart Lock',)
Product: ('GoControl Security Panel', 'GoControl Panel')
Product: ('Honeywell Smoke Detector', 'Smoke Detector', 'Living Room Smoke Detector')
Product: ('Garage Door Sensor', 'Window Sensor', 'Front Door Sensor Service', 'Motion Detector', 'Security Camera, Door/Window Sensor, Motion Detector', 'Motion D